In [13]:
using AirBorne.ETL.YFinance: get_interday_data, get_chart_data, parse_intraday_raw_data
using Dates: DateTime,datetime2unix
using Statistics
using DirectSearch
using DotMaps
using Suppressor

using AirBorne.Engines.DEDS: run
using AirBorne.Markets.StaticMarket: execute_orders!, expose_data, Order, place_order!, executeOrder_CA!
using DataFrames: DataFrame, groupby, combine, mean
using DataFrames

using AirBorne.Structures: summarizePerformance,TimeEvent, ContextTypeA
using Dates
using DirectSearch
using PaddedViews

# To generate this data use:
unix(x) = string(round(Int, datetime2unix(DateTime(x))))
tickers = ["JPM", "AAPL"]# , "MSFT", "GOOGL", "AMZN", "TSLA", "NFLX", "NVDA", "PYPL", "META"]
stocks = get_interday_data(tickers, unix("2021-01-01"), unix("2024-01-01"))

stocks5m = DataFrames.DataFrame()
for t in tickers
    data = parse_intraday_raw_data(get_chart_data(t, unix("2024-04-01"), unix("2024-04-30"), "5m"))
    # data.symbol .= t
    stocks5m = DataFrames.vcat(stocks5m, data)
end
# println(stocks5m)
# aapl = parse_intraday_raw_data(get_chart_data("AAPL", unix("2024-04-01"), unix("2024-04-30"), "5m"))

# println(stocks)

function regress(ts, lookback=1, lookahead=1)
    num_samples = length(ts) - lookback - lookahead + 1
    inputs = zeros(num_samples, lookback)
    outputs = zeros(num_samples, lookahead)
    for i  in 1:num_samples
        inputs[i,:] = (ts[i:i+lookback-1]) 
        outputs[i,:] = ts[i+lookback:i+lookback+lookahead-1]
    end
    params = inputs \ outputs 
    return params
end

function my_genOrder(
    assetId::Union{String,Symbol},
    amount::Real;
    account::Any=nothing,
    orderType::String="MarketOrder",
)
    market, ticker = split(String(assetId), "/")
    order_specs = DotMap(Dict())
    order_specs.ticker = String(ticker)
    order_specs.shares = amount # Number of shares to buy/sell
    order_specs.type = orderType
    if !(isnothing(account))
        order_specs.account = account
    end
    return Order(String(market), order_specs)
end
# function my_ordersForPortfolioRedistribution(
#     sourcePortfolio::Dict{String,Float64},
#     targetDistribution::Dict{String,Float64},
#     assetPricing::Dict{String,Float64};
#     curency_symbol::String="FEX/USD",
#     account::Any=nothing,
#     costPropFactor::Real=0,
#     costPerTransactionFactor::Real=0,
#     min_shares_threshold::Real=10^-5
# )
#     # Generate Source Distribution from Portfolio
#     sourceDst = sourcePortfolio

#     assetSort = [x for x in keys(sourceDst)]
#     N = length(assetSort)
#     curency_pos = findall(x -> x == curency_symbol, assetSort)[1]
#     ShareVals = [assetPricing[x] for x in assetSort]
#     propShareVal = ShareVals ./ totalValue # Share Price expressed in terms of portfolio units.

#     # Problem Vectorization: D1 + P*d - Fees -> D2*k
#     D1 = [get(sourceDst, x, 0) for x in assetSort] # Source
#     D2 = [get(targetDistribution, x, 0) for x in assetSort] # Objective
#     M = zeros(N, N)
#     M[curency_pos, :] = propShareVal .* -1 # Price to pay per share (without fees)
#     P = spdiagm(0 => propShareVal) + M
#     FDollars = SparseVector(N, [curency_pos], [1]) # Dollar Fees Vector

#     #####
#     ##### Optimization Problem
#     #####
#     genOrderModel = Model(Ipopt.Optimizer)
#     set_silent(genOrderModel)
#     @variable(genOrderModel, 0 <= k) # Proportionality factor (shrinkage of portfolio)
#     @variable(genOrderModel, d[1:N])  # Amount to buy/sell of each asset
#     @variable(genOrderModel, propFees >= 0) # Amount Proportional Fees
#     @constraint(
#         genOrderModel,
#         [propFees; (propShareVal .* d) .* costPropFactor] in MOI.NormOneCone(1 + N)
#     ) # Implementation of norm-1 for Fees
#     @variable(genOrderModel, perTransactionFixFees >= 0) # Number of transactions fees
#     @constraint(
#         genOrderModel, perTransactionFixFees == sum(-δ.(d) .+ 1) * costPerTransactionFactor
#     ) # Implementation of norm-1 for Fees
#     @constraint(genOrderModel, d[curency_pos] == 0) # Do not buy or sell dollars (this is the currency).
#     @constraint(
#         genOrderModel,
#         D1 .+ (P * d) .- (FDollars .* (propFees + perTransactionFixFees)) .== D2 .* k
#     ) # Distribution ratio
#     @objective(genOrderModel, Max, k) # With variance minimization
#     optimize!(genOrderModel)
#     d = value.(d)

#     #### 
#     #### Parsing & Order Generation
#     ####
#     n_shares = Dict([assetSort[x] => d[x] for x in 1:N if (x != curency_pos) && (abs(d[x])>min_shares_threshold)])
#     orders = [my_genOrder(x, n_shares[x]; account=account) for x in keys(n_shares)]
#     return orders
# end


function algo_initialize!(
    context::ContextTypeA;
    initialCapital::Real=10^5,
    nextEventFun::Union{Function,Nothing}=nothing,
    lookahead::Int=1,
    lpm_order::Float64=2.0,
    max_lookback::Int=100,
    linear_lookback::Int=1,
    tickers::Vector{String}=["^GSPC"],
    transactionCost::Real=0.001,
)
    context.extra.lookahead = lookahead
    context.extra.lpm_order = lpm_order
    context.extra.max_lookback = max_lookback
    context.extra.linear_lookback = linear_lookback
    context.extra.htcounter = 0
    context.extra.tickers = tickers
    context.extra.timecounter = 0
    context.extra.transactionCost = transactionCost
    context.extra.current_prices = Dict()

    ###################################
    ####  Specify Data Structure  ####
    ###################################
    context.extra.prices = Dict()
    context.extra.returns = Dict()
    ###################################
    ####  Initialise Portfolio  ####
    ###################################
    context.extra.portfolio = Dict(t => round(1/length(tickers); digits=2) for t in tickers)
    context.extra.portfolio = Dict(t => 0.0 for t in tickers)

    context.extra.desired_weights = context.extra.portfolio
    ###################################
    ####  Specify Account Balance  ####
    ###################################
    context.accounts.usd = DotMap(Dict())
    context.accounts.usd.balance = initialCapital
    context.accounts.usd.currency = "USD"

    # context.portfolio["FEX/USD"] = initialCapital
    #########################################
    ####  Define first simulation event  ####
    #########################################
    if !(isnothing(nextEventFun))
        nextEventFun(context)
    end
    return nothing
end

function algo_nextEvent!(context::ContextTypeA; data=DataFrame())
    # println("Computing the LPM matrix")
    curr_date = context.current_event.date
    avail_dates = unique(stocks[!,:date] .<= curr_date)
    returns = Dict()
    for t in context.extra.tickers
        prices = data[data.symbol .== t, :close][end - context.extra.max_lookback + 1:end]
        returns[t] = diff(prices) ./ prices[1:end-1]
        context.extra.current_prices[t] = prices[end]
    end
    #Compute the LPM matrix
    lpm_matrix = zeros(length(context.extra.tickers), length(context.extra.tickers))
    semi_deviations = Dict(t => mean(abs.(min.(returns[t], 0)).^context.extra.lpm_order)^(1/2) for t in context.extra.tickers)
    sorted_tickers = sort(context.extra.tickers)
    for (i, ti) in enumerate(sorted_tickers)
        for (j, tj) in enumerate(sorted_tickers)
            dev = semi_deviations[ti] * semi_deviations[tj]
            corr = cor(returns[ti], returns[tj])
            lpm_matrix[i,j] = dev * corr
        end
    end
    # println(lpm_matrix)
    #Compute the weights
    dim = length(context.extra.tickers)
    obj(x) = x' * lpm_matrix * x
    init_point = [round(1/length(tickers); digits=2) for t in context.extra.tickers]
    # println(sum(init_point))
    weights_problem = DSProblem(dim, objective=obj, granularity=[0.01 for _ in context.extra.tickers], initial_point=init_point)
    cond1(x) = round(sum(x); digits=2) == 1.00
    cond2(x) = all(x .>= 0)
    AddExtremeConstraint(weights_problem, cond1)
    AddExtremeConstraint(weights_problem, cond2)
    @suppress Optimize!(weights_problem)
    context.extra.desired_weights = Dict(t => weights_problem.x[i] for (i,t) in enumerate(sorted_tickers))
    return nothing
end

function algo_trading_logic!(
    context::ContextTypeA, data::DataFrame; nextEventFun::Union{Function,Nothing}=nothing
)
    if context.extra.timecounter < context.extra.max_lookback
        context.extra.timecounter += 1
        return nothing
    end
    if context.extra.htcounter == 0
        algo_nextEvent!(context; data=data)
        println("Portfolio: ", context.portfolio)
        println("Desired Weights: ", context.extra.desired_weights)
        println("usd balance: ", context.accounts.usd.balance)
        
        #Generate orders
        orders = []
        for t in context.extra.tickers
            assetID = String(data[data.symbol .== t, :assetID][1])
            context.extra.portfolio[t] = get(context.portfolio, assetID, 0.0) / context.accounts.usd.balance
            spend_amount = (context.extra.desired_weights[t] - context.extra.portfolio[t]) * context.accounts.usd.balance
            println("original weight: ", context.extra.portfolio[t])
            println("Spend amount: ", spend_amount)
            amount = spend_amount / context.extra.current_prices[t]
            if abs(amount) > 0
                # push!(orders, my_genOrder(assetID, amount * (1 - context.extra.transactionCost)))
                println("Buying ", amount, " of ", t, " at ", context.extra.current_prices[t])
                push!(orders, my_genOrder(assetID, amount; account=context.accounts.usd))
            end
            # assume order gets filled
        end
        println("Orders: ", orders)

        #Compute holding time
        holding_times = Dict()
        best_returns = Dict()

        for t in context.extra.tickers
            prices = data[data.symbol .== t, :close]
            # println(prices)
            # println(context.extra.linear_lookback)
            # println(context.extra.lookahead)
            params = regress(prices, context.extra.linear_lookback, context.extra.lookahead)
            forecast = prices[end-context.extra.linear_lookback+1:end]' * params
            relative_returns = log.(forecast ./ prices[end])
            holding_times[t] = argmax(collect(Iterators.flatten(relative_returns)))
            best_returns[t] = maximum(relative_returns)
        end
        s = sum(best_returns[t] * context.extra.desired_weights[t] for t in context.extra.tickers)
        # println(s)
        holding_time = sum((holding_times[t] * best_returns[t] * context.extra.desired_weights[t]) / s for t in context.extra.tickers)
        # println(holding_time)
        context.extra.htcounter = round(Int, holding_time)
        [place_order!(context, order) for order in orders]
        return nothing
    else
        context.extra.htcounter -= 1
        return nothing
    end

end



algo_trading_logic! (generic function with 1 method)

In [15]:
using Plots

evaluationEvents =[
    TimeEvent(t, "data_transfer") for t in sort(unique(stocks.date); rev=true)
]



my_init!(context) = algo_initialize!(context; lookahead=4, lpm_order=2.0, max_lookback=50, linear_lookback=2, tickers=tickers, transactionCost=0.00, initialCapital=10^3)
my_logic!(context, data) = algo_trading_logic!(context, data)
feeStructure=Vector{Dict}([Dict("FeeName" => "SaleCommission", "fixedPrice" => 0.0, "variableRate" => 0.02)])
singleExecutionFun(context, order, data) = executeOrder_CA!(context, order, data;defaultFeeStructures=feeStructure,partialExecutionAllowed=false)
my_execute_orders!(context, data) = execute_orders!(context, data; propagateBalanceToPortfolio=true, executeOrder=singleExecutionFun)
my_context = run(
    stocks,
    my_init!,
    my_logic!,
    my_execute_orders!,
    expose_data;
    audit=true,
    verbose=true,
    initialEvents=evaluationEvents,
)

dollar_symbol = "FEX/USD"

usdData = deepcopy(stocks[stocks.symbol .== my_context.extra.tickers[1], :])
usdData[!, "assetID"] .= dollar_symbol
usdData[!, "exchangeName"] .= "FEX"
usdData[!, "symbol"] .= "USD"
usdData[!, [:close, :high, :low, :open]] .= 1.0
usdData[!, [:volume]] .= 0
OHLCV_data = vcat(stocks, usdData)

println(my_context.audit.portfolioHistory)

results = summarizePerformance(stocks, my_context; includeAccounts=true)
plot(results.dollarValue, label="Dollar Value")

KeyError: KeyError: key "USD" not found

In [5]:
f(a, b; c=0) = a * b + c
g(x, y; z=2, t=0) = (x + y + z) * t
params = [2.0, 3.0]

println(f(1.0, params...))
println(g(1.0, 3.0, params...))

5.0


MethodError: MethodError: no method matching g(::Float64, ::Float64, ::Float64, ::Float64)

Closest candidates are:
  g(::Any, ::Any; z, t)
   @ Main ~/Documents/uni-4/FYP/fyp_repo/algo.ipynb:2
